# Artificial Intelligence Engineer Nanodegree - Probabilistic Models
## Project: Sign Language Recognition System
- [Introduction](#intro)
- [Part 1 Feature Selection](#part1_tutorial)
    - [Tutorial](#part1_tutorial)
    - [Features Submission](#part1_submission)
    - [Features Unittest](#part1_test)
- [Part 2 Train the models](#part2_tutorial)
    - [Tutorial](#part2_tutorial)
    - [Model Selection Score Submission](#part2_submission)
    - [Model Score Unittest](#part2_test)
- [Part 3 Build a Recognizer](#part3_tutorial)
    - [Tutorial](#part3_tutorial)
    - [Recognizer Submission](#part3_submission)
    - [Recognizer Unittest](#part3_test)
- [Part 4 (OPTIONAL) Improve the WER with Language Models](#part4_info)

<a id='intro'></a>
## Introduction
The overall goal of this project is to build a word recognizer for American Sign Language video sequences, demonstrating the power of probabalistic models.  In particular, this project employs  [hidden Markov models (HMM's)](https://en.wikipedia.org/wiki/Hidden_Markov_model) to analyze a series of measurements taken from videos of American Sign Language (ASL) collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).  In this video, the right-hand x and y locations are plotted as the speaker signs the sentence.
[![ASLR demo](http://www-i6.informatik.rwth-aachen.de/~dreuw/images/demosample.png)](https://drive.google.com/open?id=0B_5qGuFe-wbhUXRuVnNZVnMtam8)

The raw data, train, and test sets are pre-defined.  You will derive a variety of feature sets (explored in Part 1), as well as implement three different model selection criterion to determine the optimal number of hidden states for each word model (explored in Part 2). Finally, in Part 3 you will implement the recognizer and compare the effects the different combinations of feature sets and model selection criteria.  

At the end of each Part, complete the submission cells with implementations, answer all questions, and pass the unit tests.  Then submit the completed notebook for review!

<a id='part1_tutorial'></a>
## PART 1: Data

### Features Tutorial
##### Load the initial database
A data handler designed for this database is provided in the student codebase as the `AslDb` class in the `asl_data` module.  This handler creates the initial [pandas](http://pandas.pydata.org/pandas-docs/stable/) dataframe from the corpus of data included in the `data` directory as well as dictionaries suitable for extracting data in a format friendly to the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/) library.  We'll use those to create models in Part 2.

To start, let's set up the initial database and select an example set of features for the training set.  At the end of Part 1, you will create additional feature sets for experimentation. 

In [1]:
import numpy as np
import pandas as pd
from asl_data import AslDb

asl = AslDb() # initializes the database
asl.df.head() # displays the first five rows of the asl database, indexed by video and frame

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker
video frame                                                           
98    0         149     181      170      175     161      62  woman-1
      1         149     181      170      175     161      62  woman-1
      2         149     181      170      175     161      62  woman-1
      3         149     181      170      175     161      62  woman-1
      4         149     181      170      175     161      62  woman-1

In [2]:
asl.df.ix[98,1]  # look at the data available for an individual frame

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


left-x         149
left-y         181
right-x        170
right-y        175
nose-x         161
nose-y          62
speaker    woman-1
Name: (98, 1), dtype: object

The frame represented by video 98, frame 1 is shown here:
![Video 98](http://www-i6.informatik.rwth-aachen.de/~dreuw/database/rwth-boston-104/overview/images/orig/098-start.jpg)

##### Feature selection for training the model
The objective of feature selection when training a model is to choose the most relevant variables while keeping the model as simple as possible, thus reducing training time.  We can use the raw features already provided or derive our own and add columns to the pandas dataframe `asl.df` for selection. As an example, in the next cell a feature named `'grnd-ry'` is added. This feature is the difference between the right-hand y value and the nose y value, which serves as the "ground" right y value. 

In [3]:
asl.df['grnd-ry'] = asl.df['right-y'] - asl.df['nose-y']
asl.df.head()  # the new feature 'grnd-ry' is now in the frames dictionary

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  
video frame           
98    0          113  
      1          113  
      2          113  
      3          113  
      4          113

##### Try it!

In [4]:
from asl_utils import test_features_tryit
# TODO add df columns for 'grnd-rx', 'grnd-ly', 'grnd-lx' representing differences between hand and nose locations
asl.df['grnd-rx'] = asl.df['right-x'] - asl.df['nose-x']
asl.df['grnd-ly'] = asl.df['left-y'] - asl.df['nose-y']
asl.df['grnd-lx'] = asl.df['left-x'] - asl.df['nose-x']

# test the code
test_features_tryit(asl)

asl.df sample


left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly  grnd-lx  
video frame                                      
98    0          113        9      119      -12  
      1          113        9      119      -12  
      2          113        9      119      -12  
      3          113        9      119      -12  
      4          113        9      119      -12

In [5]:
# collect the features into a list
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
# show a single set of features for a given (video, frame) tuple
[asl.df.ix[98,1][v] for v in features_ground]

[9, 113, -12, 119]

##### Build the training set
Now that we have a feature list defined, we can pass that list to the `build_training` method to collect the features for all the words in the training set.  Each word in the training set has multiple examples from various videos.  Below we can see the unique words that have been loaded into the training set:

In [6]:
training = asl.build_training(features_ground)
print("Training words: {}".format(training.words))

Training words: ['JOHN', 'WRITE', 'HOMEWORK', 'IX-1P', 'SEE', 'YESTERDAY', 'IX', 'LOVE', 'MARY', 'CAN', 'GO', 'GO1', 'FUTURE', 'GO2', 'PARTY', 'FUTURE1', 'HIT', 'BLAME', 'FRED', 'FISH', 'WONT', 'EAT', 'BUT', 'CHICKEN', 'VEGETABLE', 'CHINA', 'PEOPLE', 'PREFER', 'BROCCOLI', 'LIKE', 'LEAVE', 'SAY', 'BUY', 'HOUSE', 'KNOW', 'CORN', 'CORN1', 'THINK', 'NOT', 'PAST', 'LIVE', 'CHICAGO', 'CAR', 'SHOULD', 'DECIDE', 'VISIT', 'MOVIE', 'WANT', 'SELL', 'TOMORROW', 'NEXT-WEEK', 'NEW-YORK', 'LAST-WEEK', 'WILL', 'FINISH', 'ANN', 'READ', 'BOOK', 'CHOCOLATE', 'FIND', 'SOMETHING-ONE', 'POSS', 'BROTHER', 'ARRIVE', 'HERE', 'GIVE', 'MAN', 'NEW', 'COAT', 'WOMAN', 'GIVE1', 'HAVE', 'FRANK', 'BREAK-DOWN', 'SEARCH-FOR', 'WHO', 'WHAT', 'LEG', 'FRIEND', 'CANDY', 'BLUE', 'SUE', 'BUY1', 'STOLEN', 'OLD', 'STUDENT', 'VIDEOTAPE', 'BORROW', 'MOTHER', 'POTATO', 'TELL', 'BILL', 'THROW', 'APPLE', 'NAME', 'SHOOT', 'SAY-1P', 'SELF', 'GROUP', 'JANA', 'TOY1', 'MANY', 'TOY', 'ALL', 'BOY', 'TEACHER', 'GIRL', 'BOX', 'GIVE2', 'GIVE3

The training data in `training` is an object of class `WordsData` defined in the `asl_data` module.  in addition to the `words` list, data can be accessed with the `get_all_sequences`, `get_all_Xlengths`, `get_word_sequences`, and `get_word_Xlengths` methods. We need the `get_word_Xlengths` method to train multiple sequences with the `hmmlearn` library.  In the following example, notice that there are two lists; the first is a concatenation of all the sequences(the X portion) and the second is a list of the sequence lengths(the Lengths portion).

In [7]:
training.get_word_Xlengths('CHOCOLATE')

(array([[-11,  48,   7, 120],
        [-11,  48,   8, 109],
        [ -8,  49,  11,  98],
        [ -7,  50,   7,  87],
        [ -4,  54,   7,  77],
        [ -4,  54,   6,  69],
        [ -4,  54,   6,  69],
        [-13,  52,   6,  69],
        [-13,  52,   6,  69],
        [ -8,  51,   6,  69],
        [ -8,  51,   6,  69],
        [ -8,  51,   6,  69],
        [ -8,  51,   6,  69],
        [ -8,  51,   6,  69],
        [-10,  59,   7,  71],
        [-15,  64,   9,  77],
        [-17,  75,  13,  81],
        [ -4,  48,  -4, 113],
        [ -2,  53,  -4, 113],
        [ -4,  55,   2,  98],
        [ -4,  58,   2,  98],
        [ -1,  59,   2,  89],
        [ -1,  59,  -1,  84],
        [ -1,  59,  -1,  84],
        [ -7,  63,  -1,  84],
        [ -7,  63,  -1,  84],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ -7,  63,   3,  83],
        [ 

###### More feature sets
So far we have a simple feature set that is enough to get started modeling.  However, we might get better results if we manipulate the raw values a bit more, so we will go ahead and set up some other options now for experimentation later.  For example, we could normalize each speaker's range of motion with grouped statistics using [Pandas stats](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) functions and [pandas groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html).  Below is an example for finding the means of all speaker subgroups.

In [8]:
df_means = asl.df.groupby('speaker').mean()


To select a mean that matches by speaker, use the pandas [map](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) method:

In [9]:
asl.df['left-x-mean']= asl.df['speaker'].map(df_means['left-x'])
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly  grnd-lx  left-x-mean  
video frame                                                   
98    0          113        9      119      -12   164.661438  
      1          113        9      119      -12   164.661438  
      2          113        9      119      -12   164.661438  
      3          113        9      119      -12   164.661438  
      4          113        9      119      -12   164.661438

##### Try it!

In [10]:
from asl_utils import test_std_tryit
# TODO Create a dataframe named `df_std` with standard deviations grouped by speaker
df_std = asl.df.groupby('speaker').std()
# test the code
test_std_tryit(df_std)

df_std


,left-x,left-y,right-x,right-y,nose-x,nose-y,grnd-ry,grnd-rx,grnd-ly,grnd-lx,left-x-mean
speaker,,,,,,,,,,,
man-1,15.154425,36.328485,18.901917,54.902340,6.654573,5.520045,53.487999,20.269032,36.572749,15.080360,0.0
woman-1,17.573442,26.594521,16.459943,34.667787,3.549392,3.538330,33.972660,16.764706,27.117393,17.328941,0.0
woman-2,15.388711,28.825025,14.890288,39.649111,4.099760,3.416167,39.128572,16.191324,29.320655,15.050938,0.0


<a id='part1_submission'></a>
### Features Implementation Submission
Implement four feature sets and answer the question that follows.
- normalized Cartesian coordinates
    - use *mean* and *standard deviation* statistics and the [standard score](https://en.wikipedia.org/wiki/Standard_score) equation to account for speakers with different heights and arm length
    
- polar coordinates
    - calculate polar coordinates with [Cartesian to polar equations](https://en.wikipedia.org/wiki/Polar_coordinate_system#Converting_between_polar_and_Cartesian_coordinates)
    - use the [np.arctan2](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.arctan2.html) function and *swap the x and y axes* to move the $0$ to $2\pi$ discontinuity to 12 o'clock instead of 3 o'clock;  in other words, the normal break in radians value from $0$ to $2\pi$ occurs directly to the left of the speaker's nose, which may be in the signing area and interfere with results.  By swapping the x and y axes, that discontinuity move to directly above the speaker's head, an area not generally used in signing.

- delta difference
    - as described in Thad's lecture, use the difference in values between one frame and the next frames as features
    - pandas [diff method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.diff.html) and [fillna method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) will be helpful for this one

- custom features
    - These are your own design; combine techniques used above or come up with something else entirely. We look forward to seeing what you come up with! 
    Some ideas to get you started:
        - normalize using a [feature scaling equation](https://en.wikipedia.org/wiki/Feature_scaling)
        - normalize the polar coordinates
        - adding additional deltas


In [11]:
# adding means and stdv to asl.df
cart_means = ['left-x-mean', 'left-y-mean', 'right-x-mean', 'right-y-mean']
cart_stds = ['left-x-std', 'left-y-std', 'right-x-std', 'right-y-std']
features = ['left-x', 'left-y', 'right-x', 'right-y']

for i in range(len(cart_means)):
    asl.df[cart_means[i]] = asl.df['speaker'].map(df_means[features[i]])
    
for i in range(len(cart_stds)):
    asl.df[cart_stds[i]] = asl.df['speaker'].map(df_std[features[i]])
    
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly  grnd-lx  left-x-mean  left-y-mean  \
video frame                                                                 
98    0          113        9      119      -12   164.661438   161.271242   
      1          113        9      119      -12   164.661438   161.271242   
      2          113        9      119      -12   164.661438   161.271242   
      3          113        9      119      -12   164.661438   161.271242   
      4          113        9      119      -12   164.661438   161.271242   

             right-x-mean  right-y-mean  left-x-std  left-y-std  right-x-std  \
video frame                                                                    
98    0        151.017865    117.332462   17.573442   26.594521    16.459943   
      1        151.017865    117.332462   17.573442   26.594521    16.459943   
      2        151.017865    117.332462   17.573442   26.594521    16.459943   
      3        151.017865    117.332462   17.573442   26.594521    16.459943   
      4        151.017865    117.332462   17.573442   26.594521    16.459943   

             right-y-std  
video frame               
98    0        34.667787  
      1        34.667787  
      2        34.667787  
      3        34.667787  
      4        34.667787

In [12]:
# TODO add features for normalized by speaker values of left, right, x, y
# Name these 'norm-rx', 'norm-ry', 'norm-lx', and 'norm-ly'
# using Z-score scaling (X-Xmean)/Xstd

# df_means is dataframe with means by speaker
# df_std  is dataframe with standard deviation by speaker
# 
features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']
features_to_norm = ['right-x', 'right-y', 'left-x', 'left-y']
means_norm = ['right-x-mean', 'right-y-mean', 'left-x-mean', 'left-y-mean']
stds_norm = ['right-x-std', 'right-y-std', 'left-x-std', 'left-y-std']

for i in range(len(features_norm)):
    # asl.df['norm-rx'] = asl.df['speaker'].map(df_means['right-y'])/asl.df['speaker'].map(df_std['right-y'])
    # asl.df[features_norm[i]] = (asl.df[features_to_norm[1]] - asl.df['speaker'].map(df_means[features_to_norm[i]]))/asl.df['speaker'].map(df_std[features_to_norm[i]])
    asl.df[features_norm[i]] = (asl.df[features_to_norm[i]] - asl.df[means_norm[i]]) / asl.df[stds_norm[i]]
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly    ...     right-x-mean  right-y-mean  \
video frame                               ...                                  
98    0          113        9      119    ...       151.017865    117.332462   
      1          113        9      119    ...       151.017865    117.332462   
      2          113        9      119    ...       151.017865    117.332462   
      3          113        9      119    ...       151.017865    117.332462   
      4          113        9      119    ...       151.017865    117.332462   

             left-x-std  left-y-std  right-x-std  right-y-std   norm-rx  \
video frame                                                               
98    0       17.573442   26.594521    16.459943    34.667787  1.153232   
      1       17.573442   26.594521    16.459943    34.667787  1.153232   
      2       17.573442   26.594521    16.459943    34.667787  1.153232   
      3       17.573442   26.594521    16.459943    34.667787  1.153232   
      4       17.573442   26.594521    16.459943    34.667787  1.153232   

              norm-ry   norm-lx   norm-ly  
video frame                                
98    0      1.663433 -0.891199  0.741835  
      1      1.663433 -0.891199  0.741835  
      2      1.663433 -0.891199  0.741835  
      3      1.663433 -0.891199  0.741835  
      4      1.663433 -0.891199  0.741835  

[5 rows x 23 columns]

In [13]:
# TODO add features for polar coordinate values where the nose is the origin
# Name these 'polar-rr', 'polar-rtheta', 'polar-lr', and 'polar-ltheta'
# Note that 'polar-rr' and 'polar-rtheta' refer to the radius and angle

features_polar = ['polar-rr', 'polar-rtheta', 'polar-lr', 'polar-ltheta']

right_x = asl.df['grnd-rx']
right_y = asl.df['grnd-ry']
asl.df['polar-rr'] = np.sqrt(right_x**2 + right_y**2)
asl.df['polar-rtheta'] = np.arctan2(right_x, right_y)

left_x = asl.df['grnd-lx']
left_y = asl.df['grnd-ly']
asl.df['polar-lr'] = np.sqrt(left_x**2 + left_y**2)
asl.df['polar-ltheta'] = np.arctan2(left_x, left_y)

asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly      ...       right-x-std  \
video frame                                 ...                     
98    0          113        9      119      ...         16.459943   
      1          113        9      119      ...         16.459943   
      2          113        9      119      ...         16.459943   
      3          113        9      119      ...         16.459943   
      4          113        9      119      ...         16.459943   

             right-y-std   norm-rx   norm-ry   norm-lx   norm-ly   polar-rr  \
video frame                                                                   
98    0        34.667787  1.153232  1.663433 -0.891199  0.741835  113.35784   
      1        34.667787  1.153232  1.663433 -0.891199  0.741835  113.35784   
      2        34.667787  1.153232  1.663433 -0.891199  0.741835  113.35784   
      3        34.667787  1.153232  1.663433 -0.891199  0.741835  113.35784   
      4        34.667787  1.153232  1.663433 -0.891199  0.741835  113.35784   

             polar-rtheta    polar-lr  polar-ltheta  
video frame                                          
98    0          0.079478  119.603512     -0.100501  
      1          0.079478  119.603512     -0.100501  
      2          0.079478  119.603512     -0.100501  
      3          0.079478  119.603512     -0.100501  
      4          0.079478  119.603512     -0.100501  

[5 rows x 27 columns]

In [14]:
# TODO add features for left, right, x, y differences by one time step, i.e. the "delta" values discussed in the lecture
# Name these 'delta-rx', 'delta-ry', 'delta-lx', and 'delta-ly'

features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

for i in range(len(features_delta)):
    asl.df[features_delta[i]] = asl.df[features_to_norm[i]].diff()
    asl.df[features_delta[i]].fillna(0, inplace=True)
    
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly    ...      norm-lx   norm-ly  \
video frame                               ...                          
98    0          113        9      119    ...    -0.891199  0.741835   
      1          113        9      119    ...    -0.891199  0.741835   
      2          113        9      119    ...    -0.891199  0.741835   
      3          113        9      119    ...    -0.891199  0.741835   
      4          113        9      119    ...    -0.891199  0.741835   

              polar-rr  polar-rtheta    polar-lr  polar-ltheta  delta-rx  \
video frame                                                                
98    0      113.35784      0.079478  119.603512     -0.100501       0.0   
      1      113.35784      0.079478  119.603512     -0.100501       0.0   
      2      113.35784      0.079478  119.603512     -0.100501       0.0   
      3      113.35784      0.079478  119.603512     -0.100501       0.0   
      4      113.35784      0.079478  119.603512     -0.100501       0.0   

             delta-ry  delta-lx  delta-ly  
video frame                                
98    0           0.0       0.0       0.0  
      1           0.0       0.0       0.0  
      2           0.0       0.0       0.0  
      3           0.0       0.0       0.0  
      4           0.0       0.0       0.0  

[5 rows x 31 columns]

In [15]:
df_means = asl.df.groupby('speaker').mean()
df_stds = asl.df.groupby('speaker').std()

In [16]:
df_means.head()

,left-x,left-y,right-x,right-y,nose-x,nose-y,grnd-ry,grnd-rx,grnd-ly,grnd-lx,...,norm-lx,norm-ly,polar-rr,polar-rtheta,polar-lr,polar-ltheta,delta-rx,delta-ry,delta-lx,delta-ly
speaker,,,,,,,,,,,,,,,,,,,,,
man-1,206.248203,218.679449,155.464350,150.371031,175.031756,61.642600,88.728430,-19.567406,157.036848,31.216447,...,-3.756399e-16,-2.670722e-15,95.464412,-0.307396,161.326053,0.209727,-0.022169,0.316657,0.207010,0.272768
woman-1,164.661438,161.271242,151.017865,117.332462,162.655120,57.245098,60.087364,-11.637255,104.026144,2.006318,...,-5.621744e-16,1.797158e-16,65.125528,-0.302214,105.968447,0.060601,0.041830,-0.285621,-0.286492,-0.237255
woman-2,183.214509,176.527232,156.866295,119.835714,170.318973,58.022098,61.813616,-13.452679,118.505134,12.895536,...,-9.616316e-16,2.322051e-17,67.297449,-0.258632,120.640838,0.132688,-0.008705,-0.179688,-0.015179,-0.163393


In [17]:
# TODO add features of your own design, which may be a combination of the above or something else
# Name these whatever you would like

# normalizing polar coordinates
# add polar means and stds to asl.df

polar_means = ['polar-lr-mean', 'polar-ltheta-mean', 'polar-rr-mean', 'polar-rtheta-mean']
polar_stds = ['polar-lr-std', 'polar-ltheta-std', 'polar-rr-std', 'polar-rtheta-std']
polar_features = ['polar-lr', 'polar-ltheta', 'polar-rr', 'polar-rtheta']
polar_norm = ['norm-lr', 'norm-ltheta', 'norm-rr', 'norm-rtheta']

for i in range(len(polar_means)):
    asl.df[polar_means[i]] = asl.df['speaker'].map(df_means[polar_features[i]])
    
for i in range(len(polar_stds)):
    asl.df[polar_stds[i]] = asl.df['speaker'].map(df_stds[polar_features[i]])
    
for i in range(len(polar_norm)):
    asl.df[polar_norm[i]] = (asl.df[polar_features[i]] - asl.df[polar_means[i]]) / asl.df[polar_stds[i]]    

# asl.df.drop(means+stds)
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly     ...       polar-rr-mean  \
video frame                                ...                       
98    0          113        9      119     ...           65.125528   
      1          113        9      119     ...           65.125528   
      2          113        9      119     ...           65.125528   
      3          113        9      119     ...           65.125528   
      4          113        9      119     ...           65.125528   

             polar-rtheta-mean  polar-lr-std  polar-ltheta-std  polar-rr-std  \
video frame                                                                    
98    0              -0.302214     25.133829          0.219121     30.654063   
      1              -0.302214     25.133829          0.219121     30.654063   
      2              -0.302214     25.133829          0.219121     30.654063   
      3              -0.302214     25.133829          0.219121     30.654063   
      4              -0.302214     25.133829          0.219121     30.654063   

             polar-rtheta-std   norm-lr  norm-ltheta   norm-rr  norm-rtheta  
video frame                                                                  
98    0              0.397837  0.542499    -0.735219  1.573439     0.959419  
      1              0.397837  0.542499    -0.735219  1.573439     0.959419  
      2              0.397837  0.542499    -0.735219  1.573439     0.959419  
      3              0.397837  0.542499    -0.735219  1.573439     0.959419  
      4              0.397837  0.542499    -0.735219  1.573439     0.959419  

[5 rows x 43 columns]

In [18]:
# normalized delta cartesian

norm_delta_cartesian = ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']
features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']

for i in range(len(norm_delta_cartesian)):
    asl.df[norm_delta_cartesian[i]] = asl.df[features_norm[i]].diff()
    asl.df[norm_delta_cartesian[i]].fillna(0, inplace=True)
    
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly    ...      polar-rr-std  \
video frame                               ...                     
98    0          113        9      119    ...         30.654063   
      1          113        9      119    ...         30.654063   
      2          113        9      119    ...         30.654063   
      3          113        9      119    ...         30.654063   
      4          113        9      119    ...         30.654063   

             polar-rtheta-std   norm-lr  norm-ltheta   norm-rr  norm-rtheta  \
video frame                                                                   
98    0              0.397837  0.542499    -0.735219  1.573439     0.959419   
      1              0.397837  0.542499    -0.735219  1.573439     0.959419   
      2              0.397837  0.542499    -0.735219  1.573439     0.959419   
      3              0.397837  0.542499    -0.735219  1.573439     0.959419   
      4              0.397837  0.542499    -0.735219  1.573439     0.959419   

             ndelta-rx  ndelta-ry  ndelta-lx  ndelta-ly  
video frame                                              
98    0            0.0        0.0        0.0        0.0  
      1            0.0        0.0        0.0        0.0  
      2            0.0        0.0        0.0        0.0  
      3            0.0        0.0        0.0        0.0  
      4            0.0        0.0        0.0        0.0  

[5 rows x 47 columns]

In [19]:
# normalized delta polar

norm_delta_polar = ['ndelta-rr', 'ndelta-rtheta', 'ndelta-lr', 'ndelta-ltheta']
polar_norm = ['norm-rr', 'norm-rtheta', 'norm-lr', 'norm-ltheta']

for i in range(len(norm_delta_cartesian)):
    asl.df[norm_delta_polar[i]] = asl.df[polar_norm[i]].diff()
    asl.df[norm_delta_polar[i]].fillna(0, inplace=True)
    
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly      ...         norm-rr  norm-rtheta  \
video frame                                 ...                                
98    0          113        9      119      ...        1.573439     0.959419   
      1          113        9      119      ...        1.573439     0.959419   
      2          113        9      119      ...        1.573439     0.959419   
      3          113        9      119      ...        1.573439     0.959419   
      4          113        9      119      ...        1.573439     0.959419   

             ndelta-rx  ndelta-ry  ndelta-lx  ndelta-ly  ndelta-rr  \
video frame                                                          
98    0            0.0        0.0        0.0        0.0        0.0   
      1            0.0        0.0        0.0        0.0        0.0   
      2            0.0        0.0        0.0        0.0        0.0   
      3            0.0        0.0        0.0        0.0        0.0   
      4            0.0        0.0        0.0        0.0        0.0   

             ndelta-rtheta  ndelta-lr  ndelta-ltheta  
video frame                                           
98    0                0.0        0.0            0.0  
      1                0.0        0.0            0.0  
      2                0.0        0.0            0.0  
      3                0.0        0.0            0.0  
      4                0.0        0.0            0.0  

[5 rows x 51 columns]

In [20]:
# deleting useless data from asl.df
features_to_drop = [cart_means, cart_stds, polar_means, polar_stds]
for features in features_to_drop:
    for feature in features:
        del(asl.df[feature])

asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly      ...         norm-rr  norm-rtheta  \
video frame                                 ...                                
98    0          113        9      119      ...        1.573439     0.959419   
      1          113        9      119      ...        1.573439     0.959419   
      2          113        9      119      ...        1.573439     0.959419   
      3          113        9      119      ...        1.573439     0.959419   
      4          113        9      119      ...        1.573439     0.959419   

             ndelta-rx  ndelta-ry  ndelta-lx  ndelta-ly  ndelta-rr  \
video frame                                                          
98    0            0.0        0.0        0.0        0.0        0.0   
      1            0.0        0.0        0.0        0.0        0.0   
      2            0.0        0.0        0.0        0.0        0.0   
      3            0.0        0.0        0.0        0.0        0.0   
      4            0.0        0.0        0.0        0.0        0.0   

             ndelta-rtheta  ndelta-lr  ndelta-ltheta  
video frame                                           
98    0                0.0        0.0            0.0  
      1                0.0        0.0            0.0  
      2                0.0        0.0            0.0  
      3                0.0        0.0            0.0  
      4                0.0        0.0            0.0  

[5 rows x 35 columns]

In [21]:
# TODO define a list named 'features_custom' for building the training set

features_custom = ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']

**Question 1:**  What custom features did you choose for the features_custom set and why?

**Answer 1:** The chosen faetures were norm_delta_cartesian. The eperimental reason is the higher LogL value generated when run in simulations with other features set. This result is not unanticipate, delta features is known to encode more information about sign language movements than fixed points data (Starner 1998), and between data from coordinate value itself the normalized version performed better, since the data becames more person independent. The combination of the two creates a feature set that captures movement and is person independent.

Another expected faeture set was norm_delta_polar for the same reasons as the other, and indeed it outperformed evert other feature set but the chosen one.

<a id='part1_test'></a>
### Features Unit Testing
Run the following unit tests as a sanity check on the defined "ground", "norm", "polar", and 'delta"
feature sets.  The test simply looks for some valid values but is not exhaustive.  However, the project should not be submitted if these tests don't pass.

In [157]:
import unittest
# import numpy as np

class TestFeatures(unittest.TestCase):

    def test_features_ground(self):
        sample = (asl.df.ix[98, 1][features_ground]).tolist()
        self.assertEqual(sample, [9, 113, -12, 119])

    def test_features_norm(self):
        sample = (asl.df.ix[98, 1][features_norm]).tolist()
        np.testing.assert_almost_equal(sample, [ 1.153,  1.663, -0.891,  0.742], 3)

    def test_features_polar(self):
        sample = (asl.df.ix[98,1][features_polar]).tolist()
        np.testing.assert_almost_equal(sample, [113.3578, 0.0794, 119.603, -0.1005], 3)
        
    def test_features_delta(self):
        sample = (asl.df.ix[98, 0][features_delta]).tolist()
        self.assertEqual(sample, [0, 0, 0, 0])
        sample = (asl.df.ix[98, 18][features_delta]).tolist()
        self.assertTrue(sample in [[-16, -5, -2, 4], [-14, -9, 0, 0]], "Sample value found was {}".format(sample))
                         
suite = unittest.TestLoader().loadTestsFromModule(TestFeatures())
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.011s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

<a id='part2_tutorial'></a>
## PART 2: Model Selection
### Model Selection Tutorial
The objective of Model Selection is to tune the number of states for each word HMM prior to testing on unseen data.  In this section you will explore three methods: 
- Log likelihood using cross-validation folds (CV)
- Bayesian Information Criterion (BIC)
- Discriminative Information Criterion (DIC) 

##### Train a single word
Now that we have built a training set with sequence data, we can "train" models for each word.  As a simple starting example, we train a single word using Gaussian hidden Markov models (HMM).   By using the `fit` method during training, the [Baum-Welch Expectation-Maximization](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) (EM) algorithm is invoked iteratively to find the best estimate for the model *for the number of hidden states specified* from a group of sample seequences. For this example, we *assume* the correct number of hidden states is 3, but that is just a guess.  How do we know what the "best" number of states for training is?  We will need to find some model selection technique to choose the best parameter.

In [23]:
import warnings
from hmmlearn.hmm import GaussianHMM

def train_a_word(word, num_hidden_states, features):
    
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    training = asl.build_training(features)  # WordsData object
    X, lengths = training.get_word_Xlengths(word)
    model = GaussianHMM(n_components=num_hidden_states, n_iter=1000).fit(X, lengths)
    logL = model.score(X, lengths)
    return model, logL

demoword = 'BOOK'
model, logL = train_a_word(demoword, 3, features_ground)
print("Number of states trained in model for {} is {}".format(demoword, model.n_components))
print("logL = {}".format(logL))

Number of states trained in model for BOOK is 3
logL = -2331.1138127433233


The HMM model has been trained and information can be pulled from the model, including means and variances for each feature and hidden state.  The [log likelihood](http://math.stackexchange.com/questions/892832/why-we-consider-log-likelihood-instead-of-likelihood-in-gaussian-distribution) for any individual sample or group of samples can also be calculated with the `score` method.

In [24]:
def show_model_stats(word, model):
    print("Number of states trained in model for {} is {}".format(word, model.n_components))    
    variance=np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])
    for i in range(model.n_components):  # for each hidden state
        print("hidden state #{}".format(i))
        print("mean = ", model.means_[i])
        print("variance = ", variance[i])
        print()
    
show_model_stats(demoword, model)

Number of states trained in model for BOOK is 3
hidden state #0
mean =  [-1.12415027 69.44164191 17.02866283 77.7231196 ]
variance =  [19.70434594 16.83041492 30.51552305 11.03678246]

hidden state #1
mean =  [-11.45300909  94.109178    19.03512475 102.2030162 ]
variance =  [ 77.403668   203.35441965  26.68898447 156.12444034]

hidden state #2
mean =  [-3.46504869 50.66686933 14.02391587 52.04731066]
variance =  [49.12346305 43.04799144 39.35109609 47.24195772]



##### Try it!
Experiment by changing the feature set, word, and/or num_hidden_states values in the next cell to see changes in values.  

In [26]:
my_testword = 'CHOCOLATE'
model, logL = train_a_word(my_testword, 8, features_custom) # Experiment here with different parameters
show_model_stats(my_testword, model)
print("logL = {}".format(logL))

Number of states trained in model for CHOCOLATE is 8
hidden state #0
mean =  [-1.85312448 -0.24577387 -0.39014263  0.22817316]
variance =  [0.00817329 0.00126057 0.0051453  0.01549703]

hidden state #1
mean =  [-0.82056099 -0.2440274  -0.49134196  0.17714994]
variance =  [0.11460041 0.01181247 0.00360059 0.00946517]

hidden state #2
mean =  [-0.90004266 -0.17910543 -0.04554294  0.53767424]
variance =  [0.00108003 0.01100833 0.01099692 0.00881116]

hidden state #3
mean =  [-1.65208958 -0.53357748 -0.18205255  0.88423623]
variance =  [0.03728608 0.04557645 0.00413895 0.0136116 ]

hidden state #4
mean =  [ 0.18302007 -0.42887848 -0.48498368  0.4286779 ]
variance =  [0.02316116 0.00350146 0.00779273 0.06963096]

hidden state #5
mean =  [-0.68825457 -0.47181173  0.18351751  0.59942231]
variance =  [0.04525453 0.08136263 0.03807506 0.01512759]

hidden state #6
mean =  [-0.43983589 -0.18177163 -0.25052603  0.62780063]
variance =  [0.0333588  0.0035393  0.00621128 0.00741521]

hidden state #7


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


In [27]:
features_concat = ['norm-lr', 'norm-ltheta', 'norm-rr', 'norm-rtheta', 'delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

##### Visualize the hidden states
We can plot the means and variances for each state and feature.  Try varying the number of states trained for the HMM model and examine the variances.  Are there some models that are "better" than others?  How can you tell?  We would like to hear what you think in the classroom online.

In [ ]:
def simulation_stats(sim_data):
    for features in sim_data:
        print("features tested: {}".format(features[1]))
        for hidden_states in features[0]:
            print("hidden states: {}".format(hidden_states[-1]))
            print("mean and std: {} and {}".format(hidden_states[0], hidden_states[1]))
        means = [features[0][i][0] for i in range(len(features[0]))]
        stds = [features[0][i][1] for i in range(len(features[0]))] 
        total_mean = np.array(means).mean()
        total_std = np.array(stds).mean()
        print("Overall mean and deviation for this set of features: {} and {}".format(total_mean, total_std))
        print('\n')

In [ ]:
polar_norm = ['norm-lr', 'norm-ltheta', 'norm-rr', 'norm-rtheta']
norm_delta_cartesian = ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']

In [ ]:
%matplotlib inline

In [ ]:
import math
from matplotlib import (cm, pyplot as plt, mlab)

def visualize(word, model):
    """ visualize the input model for a particular word """
    variance=np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])
    figures = []
    for parm_idx in range(len(model.means_[0])):
        xmin = int(min(model.means_[:,parm_idx]) - max(variance[:,parm_idx]))
        xmax = int(max(model.means_[:,parm_idx]) + max(variance[:,parm_idx]))
        fig, axs = plt.subplots(model.n_components, sharex=True, sharey=False)
        colours = cm.rainbow(np.linspace(0, 1, model.n_components))
        for i, (ax, colour) in enumerate(zip(axs, colours)):
            x = np.linspace(xmin, xmax, 100)
            mu = model.means_[i,parm_idx]
            sigma = math.sqrt(np.diag(model.covars_[i])[parm_idx])
            ax.plot(x, mlab.normpdf(x, mu, sigma), c=colour)
            ax.set_title("{} feature {} hidden state #{}".format(word, parm_idx, i))

            ax.grid(True)
        figures.append(plt)
    for p in figures:
        p.show()
        
visualize(my_testword, model)

#####  ModelSelector class
Review the `ModelSelector` class from the codebase found in the `my_model_selectors.py` module.  It is designed to be a strategy pattern for choosing different model selectors.  For the project submission in this section, subclass `SelectorModel` to implement the following model selectors.  In other words, you will write your own classes/functions in the `my_model_selectors.py` module and run them from this notebook:

- `SelectorCV `:  Log likelihood with CV
- `SelectorBIC`: BIC 
- `SelectorDIC`: DIC

You will train each word in the training set with a range of values for the number of hidden states, and then score these alternatives with the model selector, choosing the "best" according to each strategy. The simple case of training with a constant value for `n_components` can be called using the provided `SelectorConstant` subclass as follow:

In [235]:
from my_model_selectors import SelectorConstant

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
word = 'VEGETABLE' # Experiment here with different words
model = SelectorConstant(training.get_all_sequences(), training.get_all_Xlengths(), word, n_constant=3).select()
print("Number of states trained in model for {} is {}".format(word, model.n_components))

Number of states trained in model for VEGETABLE is 3


##### Cross-validation folds
If we simply score the model with the Log Likelihood calculated from the feature sequences it has been trained on, we should expect that more complex models will have higher likelihoods. However, that doesn't tell us which would have a better likelihood score on unseen data.  The model will likely be overfit as complexity is added.  To estimate which topology model is better using only the training data, we can compare scores using cross-validation.  One technique for cross-validation is to break the training set into "folds" and rotate which fold is left out of training.  The "left out" fold scored.  This gives us a proxy method of finding the best model to use on "unseen data". In the following example, a set of word sequences is broken into three folds using the [scikit-learn Kfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) class object. When you implement `SelectorCV`, you will use this technique.

In [236]:
from sklearn.model_selection import KFold

training = asl.build_training(features_ground) # Experiment here with different feature sets
word = 'VEGETABLE' # Experiment here with different words
word_sequences = training.get_word_sequences(word)
split_method = KFold()
for cv_train_idx, cv_test_idx in split_method.split(word_sequences):
    print("Train fold indices:{} Test fold indices:{}".format(cv_train_idx, cv_test_idx))  # view indices of the folds

Train fold indices:[2 3 4 5] Test fold indices:[0 1]
Train fold indices:[0 1 4 5] Test fold indices:[2 3]
Train fold indices:[0 1 2 3] Test fold indices:[4 5]


In [240]:
print(len(word_sequences[0][0]))

4


**Tip:** In order to run `hmmlearn` training using the X,lengths tuples on the new folds, subsets must be combined based on the indices given for the folds.  A helper utility has been provided in the `asl_utils` module named `combine_sequences` for this purpose.

##### Scoring models with other criterion
Scoring model topologies with **BIC** balances fit and complexity within the training set for each word.  In the BIC equation, a penalty term penalizes complexity to avoid overfitting, so that it is not necessary to also use cross-validation in the selection process.  There are a number of references on the internet for this criterion.  These [slides](http://www2.imm.dtu.dk/courses/02433/doc/ch6_slides.pdf) include a formula you may find helpful for your implementation.

The advantages of scoring model topologies with **DIC** over BIC are presented by Alain Biem in this [reference](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.6208&rep=rep1&type=pdf) (also found [here](https://pdfs.semanticscholar.org/ed3d/7c4a5f607201f3848d4c02dd9ba17c791fc2.pdf)).  DIC scores the discriminant ability of a training set for one word against competing words.  Instead of a penalty term for complexity, it provides a penalty if model liklihoods for non-matching words are too similar to model likelihoods for the correct word in the word set.

<a id='part2_submission'></a>
### Model Selection Implementation Submission
Implement `SelectorCV`, `SelectorBIC`, and `SelectorDIC` classes in the `my_model_selectors.py` module.  Run the selectors on the following five words. Then answer the questions about your results.

**Tip:** The `hmmlearn` library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [26]:
words_to_train = ['FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN']
import timeit

In [27]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

In [30]:
# TODO: Implement SelectorCV in my_model_selector.py
from my_model_selectors import SelectorCV

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorCV(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    try:
        if model is not None:
            print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
        else:
            print("Training failed for {}".format(word))
    except:
        print(model, type(model))

word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
word FISH has lass than 3 examples. Cannot split
(None, None) <class 'tuple'>


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encou

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encoun

Training complete for BOOK with 6 states with time 3.3314114490058273 seconds


...

Training complete for FUTURE with 2 states with time 3.1661358780111186 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Training complete for JOHN with 12 states with time 30.79168204002781 seconds


In [32]:
# TODO: Implement SelectorBIC in module my_model_selectors.py
from my_model_selectors import SelectorBIC

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorBIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Model 6 did not score for own word: FISH
Model 7 did not score for own word: FISH
Model 8 did not score for own word: FISH
Model 9 did not score for own word: FISH
Model 10 did not score for own word: FISH
Model 11 did not score for own word: FISH
Model 12 did not score for own word: FISH


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Model 13 did not score for own word: FISH
Model 14 did not score for own word: FISH
Model 15 did not score for own word: FISH
Training complete for FISH with 5 states with time 0.445737364992965 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

...

Training complete for VEGETABLE with 15 states with time 0.721308110980317 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Training complete for FUTURE with 9 states with time 1.810879475029651 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encount

Training complete for JOHN with 14 states with time 16.13180749397725 seconds


In [33]:
# TODO: Implement SelectorDIC in module my_model_selectors.py

from my_model_selectors import SelectorDIC

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorDIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH
Model did not score for own word: FISH


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


...

Training complete for VEGETABLE with 15 states with time 2.090099297987763 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encou

Training complete for FUTURE with 15 states with time 3.3849718809942715 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encount

Training complete for JOHN with 15 states with time 16.49092550104251 seconds


# Simulations for data collection and analysis

## Feature testing

In [115]:
def build_models(word_list, hmm_hidden_states, features):
    logls = []
    for word in word_list:
        model, logL = train_a_word(word, hmm_hidden_states, features) # Experiment here with different parameters
        print("logL = {} for word {} and {} hidden states".format(logL, word, hmm_hidden_states))
        logls.append(logL)
    mean_logl_state = np.array(logls).mean()
    std_logl_state = np.array(logls).std()
    return mean_logl_state, std_logl_state

def varying_hmm_states(word_list, hmm_states_list, features):
    data = []
    for hmm_state in hmm_states_list:
        print("Models with {} hidden states".format(hmm_state))
        mean_logl, std_logl = build_models(word_list, hmm_state, features)
        data.append([mean_logl, std_logl, hmm_state])
    return data

def run_simulations(word_list, hmm_hidden_list, features_lists):
    data = []
    for features in features_lists:
        print("Starting with features: {}".format(features))
        features_data = varying_hmm_states(word_list, hmm_hidden_list, features)
        data.append([features_data, features])
    return data

def simulation_stats(sim_data):
    for features in sim_data:
        print("features tested: {}".format(features[1]))
        for hidden_states in features[0]:
            print("hidden states: {}".format(hidden_states[-1]))
            print("mean and std: {} and {}".format(hidden_states[0], hidden_states[1]))
        means = [features[0][i][0] for i in range(len(features[0]))]
        stds = [features[0][i][1] for i in range(len(features[0]))] 
        total_mean = np.array(means).mean()
        total_std = np.array(stds).mean()
        print("Overall mean and deviation for this set of features: {} and {}".format(total_mean, total_std))
        print('\n')

In [116]:
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']
features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']
polar_norm = ['norm-lr', 'norm-ltheta', 'norm-rr', 'norm-rtheta']
norm_delta_cartesian = ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']
norm_delta_polar = ['ndelta-rr', 'ndelta-rtheta', 'ndelta-lr', 'ndelta-ltheta']
features_custom2 = ['norm-lr', 'norm-rr']

wordlist = ['CAN', 'WRITE', 'BLAME', 'YESTERDAY', 'CHOCOLATE'] #, 'HERE', 'THINK', 'OLD', 'SHOULD', 'FUTURE']       
hmm_list = range(4,7)
features_lists = [features_ground, features_delta, features_norm, polar_norm, norm_delta_cartesian, norm_delta_polar, features_custom2]

In [121]:
features_lists2 = [norm_delta_cartesian, norm_delta_polar, features_custom2]
run_simulations(wordlist, hmm_list, features_lists2)

Starting with features: ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']
Models with 4 hidden states
logL = 378.4930764159967 for word CAN and 4 hidden states
logL = 107.81166545493308 for word WRITE and 4 hidden states
logL = 87.7778828208825 for word BLAME and 4 hidden states
logL = 749.3854270976602 for word YESTERDAY and 4 hidden states
logL = 262.7370207859011 for word CHOCOLATE and 4 hidden states
Models with 5 hidden states
logL = 397.4898892802856 for word CAN and 5 hidden states
logL = 117.10918613637232 for word WRITE and 5 hidden states


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),


logL = 101.2617651248623 for word BLAME and 5 hidden states
logL = 727.0485440779587 for word YESTERDAY and 5 hidden states
logL = 262.79761940081573 for word CHOCOLATE and 5 hidden states
Models with 6 hidden states
logL = 425.1142097601988 for word CAN and 6 hidden states


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


logL = 124.13723233208124 for word WRITE and 6 hidden states


...

logL = 45.11234528645642 for word BLAME and 6 hidden states


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),


logL = 220.00503005590727 for word YESTERDAY and 6 hidden states
logL = 76.96620054941634 for word CHOCOLATE and 6 hidden states


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


[[[[317.24101451507477, 240.82577719195797, 4],
   [321.14140080405895, 229.76940453003283, 5],
   [349.4166904478883, 257.813150855677, 6]],
  ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']],
 [[[283.33807659118935, 187.06698873540955, 4],
   [297.81872163940363, 188.14861437979027, 5],
   [325.7004897205958, 200.54425293883503, 6]],
  ['ndelta-rr', 'ndelta-rtheta', 'ndelta-lr', 'ndelta-ltheta']],
 [[[45.45254176229513, 65.6375830205946, 4],
   [66.7199819755663, 67.75691431381745, 5],
   [80.83871076892937, 73.57098232408386, 6]],
  ['norm-lr', 'norm-rr']]]

In [262]:
%load_ext autoreload
%autoreload 2
import my_model_selectors
import importlib as imp
imp.reload(my_model_selectors)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'my_model_selectors' from '/Users/lucasfonseca/Documents/Education/Udacity/Artificial Intelligence Engineer/Foundations of AI/project-sl-recognizer/aind_p4_sign_language_recognizer/my_model_selectors.py'>

In [263]:
words_to_train = ['FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN']

## Model selectors

### SelectorCV

In [264]:
# TODO: Implement SelectorCV in my_model_selector.py
from my_model_selectors import SelectorCV

model_CV = {}
data_CV = {}
training = asl.build_training(norm_delta_cartesian)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model_CV[word], data_CV[word] = SelectorCV(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select(return_data=True)
    end = timeit.default_timer()-start
    try:
        if model is not None:
            print("Training complete for {} with {} states with time {} seconds".format(word, model_CV[word].n_components, end))
        else:
            print("Training failed for {}".format(word))
    except:
        type(type(model_CV[word]))
        print("Model for word {} was not created.".format(word))

there are 14 hidden states to be compared
Training complete for FISH with 7 states with time 0.4591339590260759 seconds
there are 14 hidden states to be compared
Training complete for BOOK with 4 states with time 4.114256325061433 seconds
there are 14 hidden states to be compared
Training complete for VEGETABLE with 2 states with time 1.7937026829458773 seconds
there are 14 hidden states to be compared
Training complete for FUTURE with 3 states with time 3.5508633160497993 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:265: RuntimeWarning: invalid value encountered in maximum
  (covars_prior + cv_num) / np.maximum(cv_den, 1e-5)


Model 14 for word JOHN did not score in split 0
No model for word JOHN and size 14
there are 13 hidden states to be compared
Training complete for JOHN with 6 states with time 202.0128224720247 seconds


In [ ]:
# for word in data_CV:
#     print(type(data_CV[word]))
#     print('Best model acc to data: {}. Model n_components: {}'.format(data_CV[word].best_hmm, model_CV[word].n_components))

print(data_CV['JOHN'][0])

In [111]:
for word in data_CV:
    print(word)
    print(type(model_CV[word]))
    try:
        print("Model size: {}. Highest mean: {}".format(data_CV[word][0], data_CV[word][1]))
    except:
        print('Word {} has no data.'.format(word))

FISH
Word FISH has no data.
BOOK
Model size: 4. Highest mean: 254.0755922829952
VEGETABLE
Model size: 2. Highest mean: 243.8929126319208
FUTURE
Model size: 3. Highest mean: 218.34565588021852
JOHN
Model size: 6. Highest mean: 3381.7713809479615


### SelectorBIC

In [107]:
from my_model_selectors import SelectorBIC

model_BIC = {}
data_BIC = {}

training = asl.build_training(norm_delta_cartesian)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    # model_BIC[word], data_BIC[word]
    model_BIC[word], data_BIC[word] = SelectorBIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select(return_data=True)
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model_BIC[word].n_components, end))
    else:
        print("Training failed for {}".format(word))

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

word: FISH. Lowest BIC = -416.1394832013745. Best number of states = 4

Training complete for FISH with 4 states with time 0.47714661195641384 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encoun

word: BOOK. Lowest BIC = -1748.6465474086942. Best number of states = 5

Training complete for BOOK with 5 states with time 2.6930801100097597 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encount

...

word: FUTURE. Lowest BIC = -1653.9682449424818. Best number of states = 5

Training complete for FUTURE with 5 states with time 2.2508797660120763 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

word: JOHN. Lowest BIC = -22604.715302912744. Best number of states = 6



/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),


Training complete for JOHN with 6 states with time 22.058440919034183 seconds


In [108]:
for word in data_BIC:
    print(word)
    print("Model size: {}. Lowest bic: {}".format(data_BIC[word][0], data_BIC[word][1]))

FISH
Model size: 4. Lowest bic: -416.1394832013745
BOOK
Model size: 5. Lowest bic: -1748.6465474086942
VEGETABLE
Model size: 2. Lowest bic: -1681.9503849312514
FUTURE
Model size: 5. Lowest bic: -1653.9682449424818
JOHN
Model size: 6. Lowest bic: -22604.715302912744


### SelectorDIC

In [105]:
from my_model_selectors import SelectorDIC

data_DIC = {}
model_DIC = {}

training = asl.build_training(norm_delta_cartesian)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()

for word in words_to_train:
    start = timeit.default_timer()
    model_DIC[word], data_DIC[word] = SelectorDIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select(True)
    end = timeit.default_timer()-start
    if model_DIC[word] is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model_DIC[word].n_components, end))
    else:
        print("Training failed for {}".format(word))

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

Training complete for FISH with 2 states with time 2.169832690968178 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encoun

Training complete for BOOK with 12 states with time 4.218913743970916 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encount

...

Training complete for VEGETABLE with 2 states with time 9.336811267014127 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

Training complete for FUTURE with 13 states with time 4.289311183965765 seconds


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encount

Training complete for JOHN with 4 states with time 25.089023270003963 seconds


In [106]:
for word in data_DIC:
    print(word)
    print("Model size: {}. Highest dic: {}".format(data_DIC[word][0], data_DIC[word][1]))
print()
for word in data_DIC:
    print()
    print(word)
    for hmm in data_DIC[word][2]:
        print('model: ', hmm[0])
        print('DIC:', hmm[1])
        

FISH
Model size: 2. Highest dic: 1224.4451209071653
BOOK
Model size: 12. Highest dic: 1011.2408309522906
VEGETABLE
Model size: 2. Highest dic: 5976.659284384235
FUTURE
Model size: 13. Highest dic: 862.3462516491295
JOHN
Model size: 4. Highest dic: 11285.283251551366


FISH
model:  2
DIC: 1224.4451209071653
model:  3
DIC: 401.7546307682302
model:  4
DIC: 1115.5876378820453
model:  5
DIC: 606.4117543717264
model:  6
DIC: 635.3962302900902
model:  7
DIC: 593.6031728227183
model:  8
DIC: 421.69898992547047
model:  9
DIC: 319.1132429010496
model:  10
DIC: 535.2485141703701
model:  11
DIC: 351.01625788355216
model:  12
DIC: 423.9024140229235
model:  13
DIC: 403.18155771669456
model:  14
DIC: 424.70624854303423
model:  15
DIC: 52.38383471791961

BOOK
model:  2
DIC: 866.0896962957347
model:  3
DIC: 887.1219796983864
model:  4
DIC: 890.7673068462688
model:  5
DIC: 906.5052694508547
model:  6
DIC: 906.7721368361645
model:  7
DIC: 920.3712134338468
model:  8
DIC: 955.6189986814313
model:  9
DIC: 

**Question 2:**  Compare and contrast the possible advantages and disadvantages of the various model selectors implemented.

**Answer 2:**  

<a id='part2_test'></a>
### Model Selector Unit Testing
Run the following unit tests as a sanity check on the implemented model selectors.  The test simply looks for valid interfaces  but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [377]:
%load_ext autoreload
%autoreload 2
import my_model_selectors
import importlib as imp
imp.reload(my_model_selectors)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'my_model_selectors' from '/Users/lucasfonseca/Documents/Education/Udacity/Artificial Intelligence Engineer/Foundations of AI/project-sl-recognizer/aind_p4_sign_language_recognizer/my_model_selectors.py'>

In [378]:
from asl_test_model_selectors import TestSelectors
suite = unittest.TestLoader().loadTestsFromModule(TestSelectors())
unittest.TextTestRunner().run(suite)

..

Best model for word John: 7


.

Word CHICKEN did not use cross validation


.
----------------------------------------------------------------------
Ran 4 tests in 46.777s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

<a id='part3_tutorial'></a>
## PART 3: Recognizer
The objective of this section is to "put it all together".  Using the four feature sets created and the three model selectors, you will experiment with the models and present your results.  Instead of training only five specific words as in the previous section, train the entire set with a feature set and model selector strategy.  
### Recognizer Tutorial
##### Train the full training set
The following example trains the entire set with the example `features_ground` and `SelectorConstant` features and model selector.  Use this pattern for you experimentation and final submission cells.



In [303]:
from my_model_selectors import SelectorConstant, SelectorBIC

def train_all_words(features, model_selector):
    training = asl.build_training(features)  # Experiment here with different feature sets defined in part 1
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    for word in training.words:
        print('Training word {}'.format(word))
        model = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select()
        model_dict[word]=model
    return model_dict

models_BIC = train_all_words(norm_delta_cartesian, SelectorBIC)
print("Number of word models returned = {}".format(len(models)))

Training word JOHN
Training word WRITE
Training word HOMEWORK
Training word IX-1P
Training word SEE
Training word YESTERDAY
Training word IX
Training word LOVE
Training word MARY
Training word CAN
Training word GO
Training word GO1
Training word FUTURE
Training word GO2
Training word PARTY
Training word FUTURE1
Training word HIT
Training word BLAME
Training word FRED
Training word FISH
Training word WONT
Training word EAT
Training word BUT
Training word CHICKEN
Training word VEGETABLE


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:265: RuntimeWarning: invalid value encountered in maximum
  (covars_prior + cv_num) / np.maximum(cv_den, 1e-5)


Training word CHINA
Training word PEOPLE
Training word PREFER
Training word BROCCOLI
Training word LIKE
Training word LEAVE
Training word SAY
Training word BUY
Training word HOUSE
Training word KNOW
Training word CORN
Training word CORN1
Training word THINK
Training word NOT
Training word PAST
Training word LIVE
Training word CHICAGO
Training word CAR
Training word SHOULD
Training word DECIDE
Training word VISIT
Training word MOVIE
Training word WANT
Training word SELL
Training word TOMORROW
Training word NEXT-WEEK
Training word NEW-YORK
Training word LAST-WEEK
Training word WILL
Training word FINISH
Training word ANN
Training word READ
Training word BOOK
Training word CHOCOLATE
Training word FIND
Training word SOMETHING-ONE
Training word POSS
Training word BROTHER
Training word ARRIVE
Training word HERE
Training word GIVE
Training word MAN
Training word NEW
Training word COAT
Training word WOMAN
Training word GIVE1
Training word HAVE
Training word FRANK
Training word BREAK-DOWN
Traini

In [281]:
print(models.keys())
print(len(list(models.keys())))

dict_keys(['JOHN', 'WRITE', 'HOMEWORK', 'IX-1P', 'SEE', 'YESTERDAY', 'IX', 'LOVE', 'MARY', 'CAN', 'GO', 'GO1', 'FUTURE', 'GO2', 'PARTY', 'FUTURE1', 'HIT', 'BLAME', 'FRED', 'FISH', 'WONT', 'EAT', 'BUT', 'CHICKEN', 'VEGETABLE', 'CHINA', 'PEOPLE', 'PREFER', 'BROCCOLI', 'LIKE', 'LEAVE', 'SAY', 'BUY', 'HOUSE', 'KNOW', 'CORN', 'CORN1', 'THINK', 'NOT', 'PAST', 'LIVE', 'CHICAGO', 'CAR', 'SHOULD', 'DECIDE', 'VISIT', 'MOVIE', 'WANT', 'SELL', 'TOMORROW', 'NEXT-WEEK', 'NEW-YORK', 'LAST-WEEK', 'WILL', 'FINISH', 'ANN', 'READ', 'BOOK', 'CHOCOLATE', 'FIND', 'SOMETHING-ONE', 'POSS', 'BROTHER', 'ARRIVE', 'HERE', 'GIVE', 'MAN', 'NEW', 'COAT', 'WOMAN', 'GIVE1', 'HAVE', 'FRANK', 'BREAK-DOWN', 'SEARCH-FOR', 'WHO', 'WHAT', 'LEG', 'FRIEND', 'CANDY', 'BLUE', 'SUE', 'BUY1', 'STOLEN', 'OLD', 'STUDENT', 'VIDEOTAPE', 'BORROW', 'MOTHER', 'POTATO', 'TELL', 'BILL', 'THROW', 'APPLE', 'NAME', 'SHOOT', 'SAY-1P', 'SELF', 'GROUP', 'JANA', 'TOY1', 'MANY', 'TOY', 'ALL', 'BOY', 'TEACHER', 'GIRL', 'BOX', 'GIVE2', 'GIVE3', 'GE

##### Load the test set
The `build_test` method in `ASLdb` is similar to the `build_training` method already presented, but there are a few differences:
- the object is type `SinglesData` 
- the internal dictionary keys are the index of the test word rather than the word itself
- the getter methods are `get_all_sequences`, `get_all_Xlengths`, `get_item_sequences` and `get_item_Xlengths`

In [282]:
test_set = asl.build_test(features_ground)
print("Number of test set items: {}".format(test_set.num_items))
print("Number of test set sentences: {}".format(len(test_set.sentences_index)))

Number of test set items: 178
Number of test set sentences: 40


<a id='part3_submission'></a>
### Recognizer Implementation Submission
For the final project submission, students must implement a recognizer following guidance in the `my_recognizer.py` module.  Experiment with the four feature sets and the three model selection methods (that's 12 possible combinations). You can add and remove cells for experimentation or run the recognizers locally in some other way during your experiments, but retain the results for your discussion.  For submission, you will provide code cells of **only three** interesting combinations for your discussion (see questions below). At least one of these should produce a word error rate of less than 60%, i.e. WER < 0.60 . 

**Tip:** The hmmlearn library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [365]:
# TODO implement the recognize method in my_recognizer
from my_recognizer import recognize
from asl_utils import show_errors
%load_ext autoreload
%autoreload 2
import my_model_selectors
import importlib as imp
imp.reload(my_model_selectors)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'my_model_selectors' from '/Users/lucasfonseca/Documents/Education/Udacity/Artificial Intelligence Engineer/Foundations of AI/project-sl-recognizer/aind_p4_sign_language_recognizer/my_model_selectors.py'>

In [ ]:
# TODO Choose a feature set and model selector
from my_model_selectors import SelectorCV, SelectorBIC, SelectorDIC
features = norm_delta_cartesian
model_selector = SelectorDIC

# TODO Recognize the test set and display the result with the show_errors method
models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

In [ ]:
# TODO Choose a feature set and model selector
from my_model_selectors import SelectorCV, SelectorBIC, SelectorDIC
features = norm_delta_cartesian
model_selector = SelectorDIC

# TODO Recognize the test set and display the result with the show_errors method
models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

In [ ]:
# TODO Choose a feature set and model selector
from my_model_selectors import SelectorCV, SelectorBIC, SelectorDIC
features = norm_delta_cartesian
model_selector = SelectorDIC

# TODO Recognize the test set and display the result with the show_errors method
models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

In [ ]:
# TODO implement the recognize method in my_recognizer
from my_recognizer import recognize
from asl_utils import show_errors
%load_ext autoreload
%autoreload 2
import my_model_selectors
import importlib as imp
imp.reload(my_model_selectors)

In [370]:
from my_model_selectors import SelectorCV, SelectorBIC, SelectorDIC

class RecognizerData(object):
    def __init__(self, features, model_selector, test_set, models, probabilities, guesses, data):
        self.features = features
        self.model_selector = model_selector
        self.probabilities = probabilities
        self.guesses = guesses
        self.test_set = test_set
        self.sim_data = data
    
def train_all_words(asl, training, features, model_selector):
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    data_dict = {}
    for word in training.words:
        model, data_dict[word] = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select(True)
        model_dict[word]=model
    return model_dict, data_dict

def multiple_recognizer(asl, features_list, model_selectors):
    data = []
    for features in features_list:
        test_set = asl.build_test(features)
        training = asl.build_training(features)
        for model_selector in model_selectors:
            # models for each word in vocab
            models , data_training= train_all_words(asl, training, features, model_selector)
            # use recognize to obtain probs and guesses
            probabilities, guesses = recognize(models, test_set)
            data.append(RecognizerData(features, model_selector, test_set, models, probabilites, guesses, data_training))
    return data

In [371]:
features_list = [norm_delta_cartesian]
model_selectors = [SelectorBIC]
recognizer_data = multiple_recognizer(asl, features_list, model_selectors)

/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:265: RuntimeWarning: invalid value encountered in maximum
  (covars_prior + cv_num) / np.maximum(cv_den, 1e-5)


In [376]:
recognizer_data[0].sim_data

{'ALL': [5,
  -276.30715613577496,
  [[2, -269.3613896162786],
   [3, -264.5093596353936],
   [4, -253.57940043764302],
   [5, -276.30715613577496],
   [6, -272.83426719854333],
   [7, -262.43714105691265],
   [8, -251.21749115309896],
   [9, -238.04769437107308],
   [10, -230.89689090884866]]],
 'ANN': [2,
  -161.68754215259003,
  [[2, -161.68754215259003],
   [3, -158.9506423570682],
   [4, -155.68215852786108],
   [5, -157.2727915068681],
   [6, -157.27269063992944],
   [7, -157.2728056792024],
   [8, -157.2728643309811],
   [9, -157.2728379827043],
   [10, -157.27286737333966]]],
 'APPLE': [3,
  -827.0324945190547,
  [[2, -740.9682633009122],
   [3, -827.0324945190547],
   [4, -748.4660530997745],
   [5, -744.6328184577526],
   [6, -787.755076634308],
   [7, -782.6978097717371],
   [8, -756.2871948540426],
   [9, -725.7078607844613],
   [10, -691.9097233543038]]],
 'ARRIVE': [6,
  -929.4786669317639,
  [[2, -716.3542468581212],
   [3, -758.8806950235635],
   [4, -771.2165134897621]

In [356]:
from my_model_selectors import SelectorCV, SelectorBIC, SelectorDIC

class RecognizerData(object):
    def __init__(self, features, model_selector, test_set, probabilities, guesses):
        self.features = features
        self.model_selector = model_selector
        self.probabilities = probabilities
        self.guesses = guesses
        self.test_set = test_set
    
def train_all_words(asl, training, features, model_selector):
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    for word in training.words:
        model = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select()
        model_dict[word]=model
    return model_dict

def multiple_recognizer(asl, features_list, model_selectors):
    data = []
    for features in features_list:
        test_set = asl.build_test(features)
        training = asl.build_training(features)
        for model_selector in model_selectors:
            # models for each word in vocab
            models = train_all_words(asl, training, features, model_selector)
            # use recognize to obtain probs and guesses
            probabilities, guesses = recognize(models, test_set)
            data.append(RecognizerData(features, model_selector, test_set, probabilites, guesses))
    return data



In [357]:
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']
features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']
polar_norm = ['norm-lr', 'norm-ltheta', 'norm-rr', 'norm-rtheta']
norm_delta_cartesian = ['ndelta-rx', 'ndelta-ry', 'ndelta-lx', 'ndelta-ly']
norm_delta_polar = ['ndelta-rr', 'ndelta-rtheta', 'ndelta-lr', 'ndelta-ltheta']
features_custom2 = ['norm-lr', 'norm-rr']

In [358]:
features_list = [norm_delta_cartesian, norm_delta_polar, polar_norm]
model_selectors = [SelectorCV, SelectorBIC, SelectorDIC]

In [359]:
features_list = [norm_delta_cartesian]
model_selectors = [SelectorCV]

In [360]:
recognizer_data = multiple_recognizer(asl, features_list, model_selectors)

Model 7 for word GO2 not working
Model 8 for word GO2 not working
Model 9 for word GO2 not working
Model 10 for word GO2 not working
Model 10 for word PARTY not working
Model 10 for word HIT not working
Model 10 for word FRED not working
Model 7 for word LIVE not working
Model 8 for word LIVE not working
Model 9 for word LIVE not working
Model 10 for word LIVE not working


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:265: RuntimeWarning: invalid value encountered in maximum
  (covars_prior + cv_num) / np.maximum(cv_den, 1e-5)


Model 10 for word SHOULD did not score in split 2
No model for word SHOULD and size 10
Model 8 for word NEXT-WEEK not working
Model 9 for word NEXT-WEEK not working
Model 10 for word NEXT-WEEK not working
Model 5 for word FIND not working
Model 6 for word FIND not working
Model 7 for word FIND not working
Model 8 for word FIND not working
Model 9 for word FIND not working
Model 10 for word FIND not working
Model 9 for word LEG not working
Model 10 for word LEG not working
Model 10 for word OLD not working
Model 7 for word BORROW not working
Model 8 for word BORROW not working
Model 9 for word BORROW not working
Model 10 for word BORROW not working


/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:251: RuntimeWarning: invalid value encountered in true_divide
  / (means_weight + denom))
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/lucasfonseca/anaconda3/envs/asl_project/lib/python3.6/site-packages/hmmlearn/hmm.py:265: RuntimeWarning: invalid value encountered in maximum
  (covars_prior + cv_num) / np.maximum(cv_den, 1e-5)


Model 10 for word BILL did not score in split 1
No model for word BILL and size 10
Model 9 for word SAY-1P not working
Model 10 for word SAY-1P not working
Model 10 for word GROUP not working
Model 10 for word MANY not working
Model 9 for word GET not working
Model 10 for word GET not working


In [362]:
# Show stats
for sim in recognizer_data:
    print(sim.guesses)
    show_errors(sim.guesses, sim.test_set)

['JOHN', 'IX', 'HOMEWORK', 'JOHN', 'STUDENT', 'GIVE1', 'TEACHER', 'JOHN', 'BOX', 'GO1', 'CAN', 'JOHN', 'FISH', 'WONT', 'MARY', 'NEW', 'NEW', 'FUTURE', 'MARY', 'JOHN', 'IX', 'YESTERDAY', 'IX', 'IX', 'JOHN', 'MARY', 'IX', 'IX', 'IX', 'JOHN', 'IX', 'IX', 'JOHN', 'IX', 'JOHN', 'IX', 'IX', 'IX', 'JOHN', 'MARY', 'JOHN', 'IX', 'JOHN', 'MARY', 'MARY', 'JOHN', 'IX', 'BUY', 'HOUSE', 'JOHN', 'IX', 'BUY', 'CAR', 'MARY', 'JOHN', 'IX', 'WHO', 'BUY', 'HOUSE', 'JOHN', 'JOHN', 'IX', 'IX', 'JOHN', 'IX', 'IX', 'BUY', 'HOUSE', 'JOHN', 'IX', 'VISIT', 'MARY', 'JOHN', 'IX', 'IX', 'MARY', 'JOHN', 'BLAME', 'MARY', 'JOHN', 'NEW', 'IX', 'BOOK', 'JOHN', 'IX', 'IX', 'IX', 'IX', 'NEW', 'CAN', 'JOHN', 'IX', 'IX', 'IX', 'IX', 'MARY', 'JOHN', 'IX', 'IX', 'IX', 'IX', 'BOY', 'JOHN', 'NEW', 'CAR', 'BREAK-DOWN', 'JOHN', 'JOHN', 'JOHN', 'JOHN', 'ARRIVE', 'YESTERDAY', 'JOHN', 'IX', 'HAVE', 'JOHN', 'CAR', 'MARY', 'IX', 'BUY1', 'IX', 'BUY1', 'IX', 'CAR', 'YESTERDAY', 'JOHN', 'PEOPLE', 'YESTERDAY', 'JOHN', 'BUY1', 'WHAT', 'JOH

**Question 3:**  Summarize the error results from three combinations of features and model selectors.  What was the "best" combination and why?  What additional information might we use to improve our WER?  For more insight on improving WER, take a look at the introduction to Part 4.

**Answer 3:**

<a id='part3_test'></a>
### Recognizer Unit Tests
Run the following unit tests as a sanity check on the defined recognizer.  The test simply looks for some valid values but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [338]:
from asl_test_recognizer import TestRecognize
suite = unittest.TestLoader().loadTestsFromModule(TestRecognize())
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 37.007s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

<a id='part4_info'></a>
## PART 4: (OPTIONAL)  Improve the WER with Language Models
We've squeezed just about as much as we can out of the model and still only get about 50% of the words right! Surely we can do better than that.  Probability to the rescue again in the form of [statistical language models (SLM)](https://en.wikipedia.org/wiki/Language_model).  The basic idea is that each word has some probability of occurrence within the set, and some probability that it is adjacent to specific other words. We can use that additional information to make better choices.

##### Additional reading and resources
- [Introduction to N-grams (Stanford Jurafsky slides)](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf)
- [Speech Recognition Techniques for a Sign Language Recognition System, Philippe Dreuw et al](https://www-i6.informatik.rwth-aachen.de/publications/download/154/Dreuw--2007.pdf) see the improved results of applying LM on *this* data!
- [SLM data for *this* ASL dataset](ftp://wasserstoff.informatik.rwth-aachen.de/pub/rwth-boston-104/lm/)

##### Optional challenge
The recognizer you implemented in Part 3 is equivalent to a "0-gram" SLM.  Improve the WER with the SLM data provided with the data set in the link above using "1-gram", "2-gram", and/or "3-gram" statistics. The `probabilities` data you've already calculated will be useful and can be turned into a pandas DataFrame if desired (see next cell).  
Good luck!  Share your results with the class!

In [ ]:
# create a DataFrame of log likelihoods for the test word items
df_probs = pd.DataFrame(data=probabilities)
df_probs.head()